# Establishing database connection, running the main query

Grouping fires by cause, creating broader subcategories

In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../data/FPA_FOD_20170508.sqlite')

query_1 = """
        SELECT
            FIRE_CODE, FIRE_NAME, FIRE_YEAR, datetime(DISCOVERY_DATE) AS DISCOVERY_DATE,
            DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_DESCR, CASE
             WHEN STAT_CAUSE_DESCR = 'Lightning' THEN 'Natural'
             WHEN STAT_CAUSE_DESCR IN ('Campfire', 'Children', 'Debris Burning', 'Equipment Use', 'Fireworks', 'Powerline', 'Railroad', 'Smoking', 'Structure') 
              THEN 'Accidental Human Activity'
             WHEN STAT_CAUSE_DESCR = 'Arson' THEN 'Intentional Human Activity'
             ELSE 'Miscellaneous/Unknown'
            END AS CAUSE_CATEGORY,
            datetime(CONT_DATE) AS CONT_DATE,
            CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, STATE, COUNTY
        FROM
            Fires
        WHERE
            COUNTY IS NOT NULL;
        """

df_1 = pd.read_sql_query(query_1, conn)

df_1.head()

,FIRE_CODE,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,CAUSE_CATEGORY,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,BJ8K,FOUNTAIN,2005,2005-02-02 00:00:00,33,1300,Miscellaneous,Miscellaneous/Unknown,2005-02-02 00:00:00,33.0,1730,0.10,A,40.036944,-121.005833,CA,63
1,AAC0,PIGEON,2004,2004-05-12 00:00:00,133,0845,Lightning,Natural,2004-05-12 00:00:00,133.0,1530,0.25,A,38.933056,-120.404444,CA,61
2,A32W,SLACK,2004,2004-05-31 00:00:00,152,1921,Debris Burning,Accidental Human Activity,2004-05-31 00:00:00,152.0,2024,0.10,A,38.984167,-120.735556,CA,17
3,None,DEER,2004,2004-06-28 00:00:00,180,1600,Lightning,Natural,2004-07-03 00:00:00,185.0,1400,0.10,A,38.559167,-119.913333,CA,3
4,None,STEVENOT,2004,2004-06-28 00:00:00,180,1600,Lightning,Natural,2004-07-03 00:00:00,185.0,1200,0.10,A,38.559167,-119.933056,CA,3


# Generating JSON file from the resulting dataframe

In [5]:
df_1.to_json("./../../frontend/public/fires.json", orient = "records", indent = 4)

# Closing the connection to the database

In [6]:
conn.close()